In [1]:
import base64
#import cStringIO
import sys
import tempfile
import time
MODEL_BASE = '/Users/aadebuger/GEXT/github2019/models/research'
MODEL_BASE = '/home/aadebugergf/Ai/github2019/models/research'


sys.path.append(MODEL_BASE)
sys.path.append(MODEL_BASE + '/object_detection')
sys.path.append(MODEL_BASE + '/slim')


import numpy as np
from PIL import Image
from PIL import ImageDraw
import tensorflow as tf
from utils import label_map_util




PATH_TO_CKPT = '/Users/aadebuger/GEXT/model/ssd_mobilenet_v1_coco_2018_01_28/frozen_inference_graph.pb'
PATH_TO_LABELS = MODEL_BASE + '/object_detection/data/mscoco_label_map.pbtxt'

content_types = {'jpg': 'image/jpeg',
                 'jpeg': 'image/jpeg',
                 'png': 'image/png'}
extensions = sorted(content_types.keys())


def is_image():
  def _is_image(form, field):
    if not field.data:
      raise ValidationError()
    elif field.data.filename.split('.')[-1].lower() not in extensions:
      raise ValidationError()

  return _is_image





class ObjectDetector(object):

  def __init__(self):
    self.detection_graph = self._build_graph()
    self.sess = tf.Session(graph=self.detection_graph)

    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    self.category_index = label_map_util.create_category_index(categories)

  def _build_graph(self):
    detection_graph = tf.Graph()
    with detection_graph.as_default():
      od_graph_def = tf.GraphDef()
      with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    return detection_graph

  def _load_image_into_numpy_array(self, image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

  def detect(self, image):
    image_np = self._load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)

    graph = self.detection_graph
    image_tensor = graph.get_tensor_by_name('image_tensor:0')
    boxes = graph.get_tensor_by_name('detection_boxes:0')
    scores = graph.get_tensor_by_name('detection_scores:0')
    classes = graph.get_tensor_by_name('detection_classes:0')
    num_detections = graph.get_tensor_by_name('num_detections:0')

    (boxes, scores, classes, num_detections) = self.sess.run(
        [boxes, scores, classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})

    boxes, scores, classes, num_detections = map(
        np.squeeze, [boxes, scores, classes, num_detections])

    return boxes, scores, classes.astype(int), num_detections


def draw_bounding_box_on_image(image, box, color='red', thickness=4):
  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  ymin, xmin, ymax, xmax = box
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  print("rect,width="+str(right-left)+"height="+str(bottom-top))
  draw.line([(left, top), (left, bottom), (right, bottom),
             (right, top), (left, top)], width=thickness, fill=color)
  print('(left, right, top, bottom)',(left, right, top, bottom))

from io import BytesIO
def encode_image(image):
  image_buffer = BytesIO()
  image.save(image_buffer, format='PNG')
  imagevalue = image_buffer.getvalue()
  base64str=base64.b64encode(imagevalue).decode()

  imgstr = 'data:image/png;base64,{:s}'.format(
     base64str )
  return imgstr


def detect_objects(image_path):
  start = time.time()
  
  image = Image.open(image_path).convert('RGB')
  boxes, scores, classes, num_detections = client.detect(image)
  end = time.time()
  print("execute1 time",(end-start)) 
    
  image.thumbnail((480, 480), Image.ANTIALIAS)
  print(num_detections)
  new_images = {}
  for i in range(int(num_detections)):

    if scores[i] < 0.35: continue
    print("scres[i",scores[i])
        
    cls = classes[i]
    if cls not in new_images.keys():
      new_images[cls] = image.copy()
    print("boxes",boxes[i])
    draw_bounding_box_on_image(new_images[cls], boxes[i],
                               thickness=int(scores[i]*10)-4)

  result = {}
#  result['original'] = encode_image(image.copy())
  print(new_images)
  count = 0 
  for cls, new_image in new_images.items():
    category = client.category_index[cls]['name']
    print(category)
    result[category] = encode_image(new_image)
    new_image.save("{0}.jpg".format(count))
    count=count+1
  end = time.time()
  print("execute2 time",(end-start)) 
  return result



PATH_TO_CKPT='/Users/aadebuger/GEXT/model/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/frozen_inference_graph.pb'
PATH_TO_CKPT='/home/aadebugergf/Ai/models/ssd_mobilenet_v1_0.75_depth_300x300_coco14_sync_2018_07_03/frozen_inference_graph.pb'

PATH_TO_CKPT='/home/aadebugergf/Ai/models/fastrcnnstbmodel1/frozen_inference_graph.pb'

client = ObjectDetector()

In [4]:
result=detect_objects('/home/aadebugergf/aixgf/tzimagedata/dqn/object20190505121605.jpg')
print(len(result))
from IPython.display import HTML
HTML("<img src='{0}'/>".format(result['book']))

execute1 time 1.5238757133483887
100.0
scres[i 0.9999925
boxes [0.50832933 0.64176506 0.81558967 0.8666642 ]
rect,width=107.95157432556152height=-110.6137204170227
(left, right, top, bottom) (308.04722785949707, 415.9988021850586, 182.99855947494507, 293.6122798919678)
{1: <PIL.Image.Image image mode=RGB size=480x360 at 0x7F5133E12908>}
person
execute2 time 1.5926926136016846
1


KeyError: 'book'

In [4]:
result=detect_objects('/home/aadebugergf/aixgf/tzimagedata/dqn/object20190505121605.jpg')
print(len(result))
from IPython.display import HTML
HTML("<img src='{0}'/>".format(result['book']))

execute1 time 0.8193366527557373
100.0
scres[i 0.9999925
boxes [0.50832933 0.64176506 0.81558967 0.8666642 ]
rect,width=107.95157432556152height=-110.6137204170227
(left, right, top, bottom) (308.04722785949707, 415.9988021850586, 182.99855947494507, 293.6122798919678)
scres[i 3.9451808e-13
scres[i 9.583658e-15
scres[i 3.1604515e-15
scres[i 2.582163e-15
scres[i 6.4195727e-16
scres[i 5.906142e-16
scres[i 4.5917836e-16
scres[i 2.3548998e-16
scres[i 1.7741574e-16
scres[i 2.0622149e-17
scres[i 1.7712418e-17
scres[i 1.7308713e-17
scres[i 1.290186e-17
scres[i 1.1884663e-17
scres[i 1.11124295e-17
scres[i 8.456729e-18
scres[i 7.750982e-18
scres[i 7.161052e-18
scres[i 6.4093383e-18
scres[i 6.0549207e-18
scres[i 5.0590755e-18
scres[i 5.02553e-18
scres[i 3.564287e-18
scres[i 3.4343379e-18
scres[i 3.4022602e-18
scres[i 2.6663522e-18
scres[i 2.5237198e-18
scres[i 2.2765493e-18
scres[i 2.0528658e-18
scres[i 1.9551553e-18
scres[i 1.7666053e-18
scres[i 1.7403316e-18
scres[i 1.7246026e-18
scres[i 1.559

KeyError: 'book'

In [6]:
import glob
itemv = glob.glob("/home/aadebugergf/aixgf/tzimagedata/dqn/object20190527*.jpg")  
for item in itemv:
        result=detect_objects(item)
        print(item)
        print(len(result))
        

execute1 time 0.8143250942230225
100.0
scres[i 0.9999896
boxes [0.56766146 0.48750418 0.86238164 0.70795524]
rect,width=105.81650733947754height=-106.09926223754883
(left, right, top, bottom) (234.00200843811035, 339.8185157775879, 204.35812711715698, 310.4573893547058)
{1: <PIL.Image.Image image mode=RGB size=480x360 at 0x7F513179A4A8>}
person
execute2 time 0.8836896419525146
/home/aadebugergf/aixgf/tzimagedata/dqn/object20190527120619.jpg
1
execute1 time 0.8150982856750488
100.0
scres[i 0.99999
boxes [0.5379058  0.48848295 0.83403426 0.70987165]
rect,width=106.26657485961914height=-106.60624265670776
(left, right, top, bottom) (234.47181701660156, 340.7383918762207, 193.64609241485596, 300.2523350715637)
{1: <PIL.Image.Image image mode=RGB size=480x360 at 0x7F513179A518>}
person
execute2 time 0.8845109939575195
/home/aadebugergf/aixgf/tzimagedata/dqn/object20190527120551.jpg
1
execute1 time 0.8231298923492432
100.0
scres[i 0.9999932
boxes [0.60207117 0.6758168  0.8884535  0.8921188 ]

execute1 time 0.8281168937683105
100.0
scres[i 0.99999154
boxes [0.50492513 0.6176208  0.8016141  0.83443785]
rect,width=104.07217025756836height=-106.80803060531616
(left, right, top, bottom) (296.4579963684082, 400.53016662597656, 181.7730474472046, 288.58107805252075)
{1: <PIL.Image.Image image mode=RGB size=480x360 at 0x7F5133E124A8>}
person
execute2 time 0.8961682319641113
/home/aadebugergf/aixgf/tzimagedata/dqn/object20190527120532.jpg
1
execute1 time 0.8296139240264893
100.0
scres[i 0.9999896
boxes [0.50133294 0.73993003 0.7950822  0.9516979 ]
rect,width=101.64856910705566height=-105.74973821640015
(left, right, top, bottom) (355.1664161682129, 456.81498527526855, 180.47985792160034, 286.2295961380005)
{1: <PIL.Image.Image image mode=RGB size=480x360 at 0x7F5133E12400>}
person
execute2 time 0.8999662399291992
/home/aadebugergf/aixgf/tzimagedata/dqn/object20190527120541.jpg
1
execute1 time 0.8297009468078613
100.0
scres[i 0.99998724
boxes [0.60003704 0.7478571  0.89277893 0.95718

execute1 time 0.8483781814575195
100.0
scres[i 0.9999889
boxes [0.44599453 0.73736006 0.74118984 0.9524719 ]
rect,width=103.25368881225586height=-106.27031207084656
(left, right, top, bottom) (353.93282890319824, 457.1865177154541, 160.55802941322327, 266.8283414840698)
{1: <PIL.Image.Image image mode=RGB size=480x360 at 0x7F5133E125C0>}
person
execute2 time 0.9199557304382324
/home/aadebugergf/aixgf/tzimagedata/dqn/object20190527120444.jpg
1
execute1 time 0.8396866321563721
100.0
scres[i 0.99999106
boxes [0.47536343 0.61746144 0.7731548  0.8345316 ]
rect,width=104.19367790222168height=-107.20489025115967
(left, right, top, bottom) (296.3814926147461, 400.5751705169678, 171.13083600997925, 278.3357262611389)
{1: <PIL.Image.Image image mode=RGB size=480x360 at 0x7F5133E12630>}
person
execute2 time 0.9087734222412109
/home/aadebugergf/aixgf/tzimagedata/dqn/object20190527120503.jpg
1
execute1 time 0.8396306037902832
100.0
scres[i 0.9999871
boxes [0.58829147 0.6775051  0.87827265 0.8907128

execute1 time 0.8430399894714355
100.0
scres[i 1.0
boxes [0. 0. 1. 1.]
rect,width=480.0height=-360.0
(left, right, top, bottom) (0.0, 480.0, 0.0, 360.0)
scres[i 1.0
boxes [0. 0. 1. 1.]
rect,width=480.0height=-360.0
(left, right, top, bottom) (0.0, 480.0, 0.0, 360.0)
scres[i 1.0
boxes [0. 0. 1. 1.]
rect,width=480.0height=-360.0
(left, right, top, bottom) (0.0, 480.0, 0.0, 360.0)
scres[i 1.0
boxes [0. 0. 1. 1.]
rect,width=480.0height=-360.0
(left, right, top, bottom) (0.0, 480.0, 0.0, 360.0)
scres[i 1.0
boxes [0. 0. 1. 1.]
rect,width=480.0height=-360.0
(left, right, top, bottom) (0.0, 480.0, 0.0, 360.0)
scres[i 1.0
boxes [0. 0. 1. 1.]
rect,width=480.0height=-360.0
(left, right, top, bottom) (0.0, 480.0, 0.0, 360.0)
scres[i 1.0
boxes [0. 0. 1. 1.]
rect,width=480.0height=-360.0
(left, right, top, bottom) (0.0, 480.0, 0.0, 360.0)
scres[i 1.0
boxes [0. 0. 1. 1.]
rect,width=480.0height=-360.0
(left, right, top, bottom) (0.0, 480.0, 0.0, 360.0)
scres[i 1.0
boxes [0. 0. 1. 1.]
rect,width=480.0h

In [8]:
import glob
itemv = glob.glob("/home/aadebugergf/aixgf/tzimagedata/object20190527*.jpg")  
for item in itemv:
        print(item)
        result=detect_objects(item)
  
        print(len(result))

/home/aadebugergf/aixgf/tzimagedata/object20190527120208.jpg
execute1 time 0.8054141998291016
100.0
{}
execute2 time 0.8108339309692383
0
/home/aadebugergf/aixgf/tzimagedata/object20190527120323.jpg
execute1 time 0.8016419410705566
100.0
{}
execute2 time 0.8070564270019531
0
/home/aadebugergf/aixgf/tzimagedata/object20190527121459.jpg
execute1 time 0.805387020111084
100.0
scres[i 0.99999857
boxes [0.4896228  0.19625632 0.7926634  0.41797563]
rect,width=106.42526865005493height=-109.09461379051208
(left, right, top, bottom) (94.20303583145142, 200.62830448150635, 176.26420855522156, 285.35882234573364)
{1: <PIL.Image.Image image mode=RGB size=480x360 at 0x7F513179AC50>}
person
execute2 time 0.867830753326416
1
/home/aadebugergf/aixgf/tzimagedata/object20190527120217.jpg
execute1 time 0.8111190795898438
100.0
{}
execute2 time 0.8165359497070312
0
/home/aadebugergf/aixgf/tzimagedata/object20190527121728.jpg
execute1 time 0.8091630935668945
100.0
scres[i 0.9999937
boxes [0.6881886  0.17724

In [2]:
import glob
itemv = glob.glob("/home/aadebugergf/aixgf/tzimagedata/object20190527*.jpg")  
for item in itemv:
        print(item)
        result=detect_objects(item)
  
        print(len(result))

/home/aadebugergf/aixgf/tzimagedata/object20190527120208.jpg
execute1 time 1.8800106048583984
100.0
{}
execute2 time 1.885174036026001
0
/home/aadebugergf/aixgf/tzimagedata/object20190527120323.jpg
execute1 time 0.8319883346557617
100.0
{}
execute2 time 0.8373847007751465
0
/home/aadebugergf/aixgf/tzimagedata/object20190527121459.jpg
execute1 time 0.8226199150085449
100.0
scres[i 0.99999857
boxes [0.4896228  0.19625632 0.7926634  0.41797563]
rect,width=106.42526865005493height=109.09461379051208
(left, right, top, bottom) (94.20303583145142, 200.62830448150635, 176.26420855522156, 285.35882234573364)
{1: <PIL.Image.Image image mode=RGB size=480x360 at 0x7F0E488D6B00>}
person
execute2 time 0.8937609195709229
1
/home/aadebugergf/aixgf/tzimagedata/object20190527120217.jpg
execute1 time 0.8591933250427246
100.0
{}
execute2 time 0.8645858764648438
0
/home/aadebugergf/aixgf/tzimagedata/object20190527121728.jpg
execute1 time 0.8426775932312012
100.0
scres[i 0.9999937
boxes [0.6881886  0.17724